In [1]:
import os
os.path.expanduser('~/.keras/models')
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
model_conv = VGG16(weights='imagenet', input_shape=(150, 150, 3), include_top=False)

In [4]:
model_conv.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0     

In [6]:
data_generator = ImageDataGenerator(
    rescale=1.0/255.0,
)
features = np.zeros(shape=(2000, 4, 4, 512))
labels = np.zeros(shape=(2000))
generator = data_generator.flow_from_directory(
    'PetImages',
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary'
)
i = 0
for inputs_batch, labels_batch in generator:
    features[i*20: (i+1)*20] = model_conv.predict(inputs_batch)
    labels[i*20: (i+1)*20] = labels_batch
    i=i+1
    if i*20 >= 2000:
        break

Found 2000 images belonging to 2 classes.


In [7]:
features.shape

(2000, 4, 4, 512)

In [8]:
labels.shape

(2000,)

In [9]:
data_generator_test = ImageDataGenerator(
    rescale=1.0/255.0,
)
features_test = np.zeros(shape=(1400, 4, 4, 512))
labels_test = np.zeros(shape=(1400))
generator_test = data_generator_test.flow_from_directory(
    'PetImagesTest/',
    target_size=(150, 150),
    batch_size=10,
    class_mode='binary'
)
i = 0
for inputs_batch, labels_batch in generator_test:
    features_test[i*10: (i+1)*10] = model_conv.predict(inputs_batch)
    labels_test[i*10: (i+1)*10] = labels_batch
    i=i+1
    if i*10 >= 1400:
        break

Found 1400 images belonging to 2 classes.


In [10]:
import gc
del model_conv
gc.collect()

25658

In [11]:
features = np.reshape(features, (2000, 4 * 4 * 512))
features_test = np.reshape(features_test, (1400, 4 * 4 * 512))
features.shape

(2000, 8192)

In [12]:
4 * 4 * 512

8192

In [13]:
import pickle
with open('vggfeatures.h5', 'wb') as f:
    pickle.dump(features, f)

In [14]:
with open('vgglabels.h5', 'wb') as f:
    pickle.dump(labels, f)

In [15]:
with open('vggfeatures_test.h5', 'wb') as f:
    pickle.dump(features_test, f)
with open('vgglabels_test.h5', 'wb') as f:
    pickle.dump(labels_test, f)